In [1]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

import datetime as dt
import requests

In [2]:
alterar='20200908'

## JHU

In [3]:
url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
df_confirmed = pd.read_csv(url, error_bad_lines=False,parse_dates=True)

url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
df_death = pd.read_csv(url, error_bad_lines=False,parse_dates=True)

## Alteração nos dados baseados nas correções dos reportes oficiais

O reporte do dia 17/02/20 (referente aos dados de 16/02/20) diz que a partir de então, os dados serão confirmados pelos exames laboratoriais e clínicos (antes era apenas laobatorial- apenas Hubei). Com isso, atualizamos os dados de 12 a 14/02 (que tem esses dados disponiveis) 

https://www.who.int/docs/default-source/coronaviruse/situation-reports/20200217-sitrep-28-covid-19.pdf?sfvrsn=a19cf2ad_2

In [4]:
# função para alterar valor
def change_val(date, ref_col, dataset,dtnry):
    for key, val in dtnry.items():
        dataset[date].loc[(dataset[ref_col]==key)]=val

In [5]:
feb_11_conf = {'Hubei' : 33366}
feb_12_conf = {'Hubei' : 34874}
feb_13_conf = {'Hubei' : 51986}
feb_14_conf = {'Hubei' : 54406}
       
change_val("2/12/20","Province/State", df_confirmed,feb_12_conf)
change_val("2/13/20","Province/State",df_confirmed,feb_13_conf)
change_val("2/14/20","Province/State",df_confirmed,feb_14_conf)

## Criando dataset diário

In [6]:
def diff_serie(dataframe : pd.DataFrame) -> pd.DataFrame:
    
    df = dataframe.copy()
    df_diff = df.iloc[:,4:]
    df_diff = df_diff.T.diff().T
    df_diff.mask(df_diff < 0, 0,inplace=True)
    df_diff=pd.concat([df.iloc[:,0:4],df_diff.iloc[:,1:]], axis=1)
          
    return df_diff

In [7]:
df_death1=diff_serie(df_death)
df_confirmed1=diff_serie(df_confirmed)

## Dataset diário no formato long

In [8]:
dates=df_confirmed1.columns[4:]

confirmed_long = df_confirmed1.melt(id_vars=['Province/State', 'Country/Region'], 
                            value_vars=dates, var_name='Date', value_name='Confirmed')

death_long = df_death1.melt(id_vars=['Province/State', 'Country/Region'], 
                            value_vars=dates, var_name='Date', value_name='Deaths')

full_long=pd.merge(confirmed_long, death_long,on=["Country/Region","Province/State","Date"], how='left')

full_long["Date"]= pd.to_datetime(full_long["Date"])
full_long["Date"] = full_long["Date"].dt.strftime('%m/%d/%Y')

# Corrigindo a nomeclatura de alguns paises
full_long['Country/Region'] = full_long['Country/Region'].replace('Korea, South', 'South Korea')
full_long['Country/Region'] = full_long['Country/Region'].replace('Taiwan*', 'Taiwan')
full_long['Country/Region'] = full_long['Country/Region'].replace('US', 'United States')

## Agrupando dataset por país

In [9]:
df=full_long.groupby(["Date",'Country/Region'])[["Date","Confirmed","Deaths"]].sum().reset_index()

In [10]:
df

,Date,Country/Region,Confirmed,Deaths
0,01/23/2020,Afghanistan,0.0,0.0
1,01/23/2020,Albania,0.0,0.0
2,01/23/2020,Algeria,0.0,0.0
3,01/23/2020,Andorra,0.0,0.0
4,01/23/2020,Angola,0.0,0.0
...,...,...,...,...
43047,09/07/2020,West Bank and Gaza,652.0,3.0
43048,09/07/2020,Western Sahara,0.0,0.0
43049,09/07/2020,Yemen,2.0,1.0
43050,09/07/2020,Zambia,60.0,0.0


## Diluindo os casos clinicos da China (pico de 15410 em 12/02/20) nos dias anteriores

In [11]:
feb_13_conf = {'China' : 1998}   
df.loc[(df["Country/Region"]=="China")& (df["Date"]=='02/13/2020') , "Confirmed"] =1998
soma=df[df["Country/Region"]=="China"][0:22].groupby(['Country/Region'])["Confirmed"].sum()[0]


_f1 = df['Country/Region'] == 'China'
_f2 = df['Date'] <= '02/13/2020'
df.loc[_f1 & _f2, 'Confirmed'] += round(df.loc[_f1 & _f2, 'Confirmed']*15410/soma)

## Dataset oxford

In [12]:
url = "https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/c1_schoolclosing.csv"
c1 = pd.read_csv(url, error_bad_lines=False,parse_dates=True, sep=",", encoding="latin",skipfooter=3, na_values=['.'])

url = "https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/c2_workplaceclosing.csv"
c2 = pd.read_csv(url, error_bad_lines=False,parse_dates=True, sep=",", encoding="latin",skipfooter=3, na_values=['.'])

url = "https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/c3_cancelpublicevents.csv"
c3 = pd.read_csv(url, error_bad_lines=False,parse_dates=True, sep=",", encoding="latin",skipfooter=3, na_values=['.'])

url = "https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/c4_restrictionsongatherings.csv"
c4 = pd.read_csv(url, error_bad_lines=False,parse_dates=True, sep=",", encoding="latin",skipfooter=3, na_values=['.'])

url = "https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/c5_closepublictransport.csv"
c5 = pd.read_csv(url, error_bad_lines=False,parse_dates=True, sep=",", encoding="latin",skipfooter=3, na_values=['.'])

url = "https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/c6_stayathomerequirements.csv"
c6 = pd.read_csv(url, error_bad_lines=False,parse_dates=True, sep=",", encoding="latin",skipfooter=3, na_values=['.'])

url = "https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/c7_domestictravel.csv"
c7 = pd.read_csv(url, error_bad_lines=False,parse_dates=True, sep=",", encoding="latin",skipfooter=3, na_values=['.'])

url = "https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/c8_internationaltravel.csv"
c8 = pd.read_csv(url, error_bad_lines=False,parse_dates=True, sep=",", encoding="latin",skipfooter=3, na_values=['.'])

In [13]:
dates=c1.columns[2:]

df_c1 = c1.melt(id_vars=['Unnamed: 0', 'Unnamed: 1'], 
                            value_vars=dates, var_name='Date', value_name='C1')
df_c2 = c2.melt(id_vars=['Unnamed: 0', 'Unnamed: 1'], 
                            value_vars=dates, var_name='Date', value_name='C2')
df_c3 = c3.melt(id_vars=['Unnamed: 0', 'Unnamed: 1'], 
                            value_vars=dates, var_name='Date', value_name='C3')
df_c4 = c4.melt(id_vars=['Unnamed: 0', 'Unnamed: 1'], 
                            value_vars=dates, var_name='Date', value_name='C4')
df_c5 = c5.melt(id_vars=['Unnamed: 0', 'Unnamed: 1'], 
                            value_vars=dates, var_name='Date', value_name='C5')
df_c6 = c6.melt(id_vars=['Unnamed: 0', 'Unnamed: 1'], 
                            value_vars=dates, var_name='Date', value_name='C6')
df_c7 = c7.melt(id_vars=['Unnamed: 0', 'Unnamed: 1'], 
                            value_vars=dates, var_name='Date', value_name='C7')
df_c8 = c8.melt(id_vars=['Unnamed: 0', 'Unnamed: 1'], 
                            value_vars=dates, var_name='Date', value_name='C8')


full=pd.merge(df_c1,df_c2,on=["Unnamed: 0","Unnamed: 1","Date"], how='left')
full=pd.merge(full,df_c3,on=["Unnamed: 0","Unnamed: 1","Date"], how='left')
full=pd.merge(full,df_c4,on=["Unnamed: 0","Unnamed: 1","Date"], how='left')
full=pd.merge(full,df_c5,on=["Unnamed: 0","Unnamed: 1","Date"], how='left')
full=pd.merge(full,df_c6,on=["Unnamed: 0","Unnamed: 1","Date"], how='left')
full=pd.merge(full,df_c7,on=["Unnamed: 0","Unnamed: 1","Date"], how='left')
full_long_mit=pd.merge(full,df_c8,on=["Unnamed: 0","Unnamed: 1","Date"], how='left')

full_long_mit["Date"]= pd.to_datetime(full_long_mit["Date"])
full_long_mit["Date"] = full_long_mit["Date"].dt.strftime('%m/%d/%Y')

full_long_mit.rename(columns={'Unnamed: 0': 'Country/Region', "Unnamed: 1":'COD'}, inplace=True)
full_long_mit=full_long_mit.drop(columns="COD")

full_long_mit['Country/Region'] = full_long_mit['Country/Region'].replace( 'Cape Verde', 'Cabo Verde')
full_long_mit['Country/Region'] = full_long_mit['Country/Region'].replace(  'Democratic Republic of Congo', 'Congo (Kinshasa)')
full_long_mit['Country/Region'] = full_long_mit['Country/Region'].replace( 'Congo',  'Congo (Brazzaville)')
full_long_mit['Country/Region'] = full_long_mit['Country/Region'].replace(  'Kyrgyz Republic',  'Kyrgyzstan')
full_long_mit['Country/Region'] = full_long_mit['Country/Region'].replace(  'Slovak Republic', 'Slovakia')

In [14]:
full_long_mit.head()

,Country/Region,Date,C1,C2,C3,C4,C5,C6,C7,C8
0,Aruba,01/01/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Afghanistan,01/01/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Angola,01/01/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Anguilla,01/01/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Albania,01/01/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
bd=full_long_mit.copy()
bd["C1"].loc[(bd["C1"]>0)]=1
bd["C2"].loc[(bd["C2"]>0)]=1
bd["C3"].loc[(bd["C3"]>0)]=1
bd["C4"].loc[(bd["C4"]>0)]=1
bd["C5"].loc[(bd["C5"]>0)]=1
bd["C6"].loc[(bd["C6"]>0)]=1
bd["C7"].loc[(bd["C7"]>0)]=1
bd["C8"].loc[(bd["C8"]>0)]=1

In [16]:
df_complete=pd.merge(df,bd, how="outer").sort_values("Date").reset_index(drop=True)
df_complete=pd.merge(df,bd,on=["Country/Region","Date"], how='left')

## Notas fiscais

In [17]:
# Lendo tabela com os dados
sefaz_df = pd.read_csv('http://receitadados.fazenda.rs.gov.br/Documentos%20Compartilhados/Valor%20e%20quantidade%20de%20DFe%20por%20dia.csv',
                       sep=';', encoding='latin-1', decimal=',',
                       parse_dates=['Data', 'Dados Atualizados Até']
                      )

# Eliminando colunas sem utilidade
sefaz_df = sefaz_df.drop(columns=['Ano', 'Mês', 'Tipo Emissão', 'Dados Atualizados Até'])

# Selecionando apenas amostras de 2019 em diante
sefaz_df = sefaz_df.loc[sefaz_df['Data'].dt.year >= 2019]

# Selecionando apenas amostras de NFC-e
sefaz_nfc_df = sefaz_df.loc[(sefaz_df['Modelo'] == 'NFC-e')]

# Eliminando coluna de modelo
sefaz_nfc_df = sefaz_nfc_df.drop(columns=['Modelo'])

# Ordenando valores pela Data
sefaz_nfc_df = sefaz_nfc_df.sort_values(by='Data')

# Substituindo nomes dos estados pelas siglas
states_alias = {'RIO GRANDE DO NORTE':'RN', 'SERGIPE':'SE', 'ESPIRITO SANTO':'ES',
                'RIO DE JANEIRO':'RJ', 'RONDONIA':'RO', 'ALAGOAS':'AL', 'RORAIMA':'RR',
                'BAHIA':'BA', 'RIO GRANDE DO SUL':'RS', 'ACRE':'AC', 'PIAUI':'PI', 'PARAIBA':'PB'}

sefaz_nfc_df['UF'] = sefaz_nfc_df['UF'].replace(states_alias)

In [18]:
sefaz_df_teste=sefaz_nfc_df.copy()
sefaz_df_teste=sefaz_df_teste.loc[sefaz_df_teste['Data'].dt.year == 2019]
sefaz_mean = sefaz_df_teste.groupby('UF').mean()
sefaz_mean = sefaz_mean.add_suffix('_2019_mean')
sefaz_mean = sefaz_mean.reset_index()
sefaz_nfc_df = sefaz_nfc_df.merge(sefaz_mean, on='UF')

## Casos confirmados brasil

In [19]:
headers = {
    'Host': 'xx9p7hp1p7.execute-api.us-east-1.amazonaws.com',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br',
    'X-Parse-Application-Id': 'unAFkcaNDeXajurGB7LChj8SgQYS2ptm',
    'Origin': 'https://covid.saude.gov.br',
    'Connection': 'keep-alive',
    'Referer': 'https://covid.saude.gov.br/',
    'TE': 'Trailers',
    }
json_with_url = requests.get('https://xx9p7hp1p7.execute-api.us-east-1.amazonaws.com/prod/PortalGeral', headers=headers)

In [20]:
daily_covid_url = json_with_url.json().get('results')[0].get('arquivo').get('url')
# covid_br_path = '../covid_br_data/HIST_PAINEL_COVIDBR_16jun2020.xlsx'

covid_br_df = pd.read_excel(daily_covid_url, parse_dates=['data'])

# Dicionário com estados que possuem dados de notas fiscais
states_alias = {'RIO GRANDE DO NORTE':'RN', 'SERGIPE':'SE', 'ESPIRITO SANTO':'ES',
                'RIO DE JANEIRO':'RJ', 'RONDONIA':'RO', 'ALAGOAS':'AL', 'RORAIMA':'RR',
                'BAHIA':'BA', 'RIO GRANDE DO SUL':'RS', 'ACRE':'AC', 'PIAUI':'PI', 'PARAIBA':'PB'}

# Selecionando apenas estados que possuem dados de notas fiscais
covid_br_df = covid_br_df[covid_br_df['estado'].isin(states_alias.values())]

# Selecionando dados relativos aos estados, desconsiderando municípios
covid_br_df = covid_br_df[covid_br_df['codmun'].isna()]

# Eliminando colunas sem utilidade
covid_br_df = covid_br_df.drop(
    columns=['regiao', 'municipio', 'coduf', 'codmun', 'codRegiaoSaude', 'nomeRegiaoSaude', 'semanaEpi',
             'Recuperadosnovos', 'emAcompanhamentoNovos', 'interior/metropolitana', 'casosAcumulado', 'obitosAcumulado']
)

# Modificando nome das colunas
covid_br_df.columns = ['UF', 'Data', 'populacao',
                       'casosNovos', 'obitosNovos']

In [21]:
UFs = set(covid_br_df.UF)

covid_br_new = pd.DataFrame(columns=covid_br_df.columns)

for uf in UFs:
    estado_df = covid_br_df[covid_br_df['UF'] == uf]

    pop = estado_df.loc[:, 'populacao'].iloc[0]

    for i in range(15, 25):
        new_row = {
            'UF': uf, 
            'Data': f'{i}/02/2020',
            'populacao': pop, 
            
            'casosNovos': 0, 
            
            'obitosNovos': 0
        }
        estado_df = estado_df.append(new_row, ignore_index=True)
    
    estado_df['Data'] = pd.to_datetime(estado_df['Data'])
    covid_br_new = covid_br_new.append(estado_df, ignore_index=True)

covid_br_new = covid_br_new.sort_values(by=['UF', 'Data'])

In [22]:
full_dataset = sefaz_nfc_df.merge(covid_br_new, how='left', on=['UF', 'Data'], validate='1:1')

## Mitigação nacional

In [23]:
mitigacao_path = './brazil_states_policies_dataset/'
df_mitigacao = pd.read_csv(mitigacao_path+'classificacoes_all_'+alterar+'.csv', sep=';', encoding='utf-16')

# adicionando dias zerados no dataset para fazer merge
for i in range(15, 25):
    df_mitigacao[f'{i}/02/2020'] = 0

dates = df_mitigacao.columns[2:]
df_mitigacao = df_mitigacao.melt(
        id_vars=['Classificacao', 'StateCode'],
        value_vars=dates,
        var_name='Date',
        value_name='Mitigacao',
    )

df_mitigacao = df_mitigacao.pivot_table(index=['StateCode', 'Date'], columns='Classificacao', fill_value=0)

df_mitigacao.columns = measures = df_mitigacao.columns.droplevel()
df_mitigacao = df_mitigacao.reset_index()
df_mitigacao.columns = ['UF', 'Data'] + list(measures)

df_mitigacao['Data'] = pd.to_datetime(df_mitigacao['Data'], format='%d/%m/%Y')
df_mitigacao = df_mitigacao.sort_values(by=['UF', 'Data'])

In [24]:
full_dataset = pd.merge(full_dataset, df_mitigacao, on=['Data', 'UF'], how='inner').sort_values(by=['UF', 'Data'])
uf_population = full_dataset.loc[:, ['UF','populacao']].drop_duplicates().dropna()

for UF, population in zip(uf_population['UF'], uf_population['populacao']):
    full_dataset.loc[full_dataset['UF']==UF, 'populacao'] = population
    
full_dataset = full_dataset.fillna(value=0)
full_dataset = full_dataset.astype({'Qtde Autorizados':float})

In [25]:
full_dataset = full_dataset.set_index('Data')
full_dataset.insert(1, 'day_of_week', full_dataset.index.dayofweek) # adição do dia da semana no dataset
full_dataset.reset_index(inplace=True)

full_dataset.rename(columns={'UF': 'Country/Region', "Data":'Date', "casosNovos":"Confirmed", "obitosNovos":"Deaths"}, inplace=True)

full_dataset["Date"]= pd.to_datetime(full_dataset["Date"])
full_dataset["Date"] = full_dataset["Date"].dt.strftime('%m/%d/%Y')

In [26]:
df_final=pd.concat([df_complete,full_dataset],axis=0, sort=True)

In [27]:
df=df_final.copy()
country=df["Country/Region"].unique()
df_model = pd.DataFrame([])

for uf in country:
    
    df1 = df[df['Country/Region']==uf].reset_index(drop=True)
    index=df1[df1["Confirmed"]>0].first_valid_index()
    data=df1.iloc[index:]
    df_model=df_model.append(data)

df_model=df_model.reset_index(drop=True)

In [28]:
df_model.to_csv('data/dados_gerais_'+ alterar +'.csv', index=False)
df_complete.to_csv('data/dados_mundiais_'+ alterar +'.csv', index=False)
full_dataset.to_csv('data/dados_nacionais_'+ alterar+'.csv', index=False)